### The Data
This book will try to develop an algorithm to identify an atoms nearby functional groups, that is, in CH3-CH2-OH the H atoms in CH3 are part of a CH3-group, neigherst neighbor are CH2 followed by OH.

Note: this book will only consider data from the following csv files: train, test and structures

In [1]:
import pandas as pd

input_folder = './input'

train = pd.read_csv(f'{input_folder}/train.csv')
structures = pd.read_csv(f'{input_folder}/structures.csv')

In [2]:
f'Train shape: {train.shape}'

'Train shape: (4658147, 6)'

In [3]:
f'Structures shape: {structures.shape}'

'Structures shape: (2358657, 6)'

In [4]:
print(f"There are {train['type'].nunique()} unique coupling types: {train['type'].unique()}")


There are 8 unique coupling types: ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']


#### Visualising a few molecules from train, structures and structured/xyz-files

In [16]:
train.loc[train['molecule_name'] == 'dsgdb9nsd_000013']

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
106,106,dsgdb9nsd_000013,3,0,1JHC,83.51320
107,107,dsgdb9nsd_000013,3,1,2JHC,-2.03004
108,108,dsgdb9nsd_000013,3,2,3JHC,2.51277
109,109,dsgdb9nsd_000013,3,4,2JHH,-11.43170
110,110,dsgdb9nsd_000013,3,5,2JHH,-11.47950
111,111,dsgdb9nsd_000013,3,6,3JHH,3.42929
112,112,dsgdb9nsd_000013,3,7,3JHH,12.33070
113,113,dsgdb9nsd_000013,4,0,1JHC,83.51430
114,114,dsgdb9nsd_000013,4,1,2JHC,-2.03044
115,115,dsgdb9nsd_000013,4,2,3JHC,2.51802


In [27]:
structures.loc[structures['molecule_name'] == 'dsgdb9nsd_000230']

,molecule_name,atom_index,atom,x,y,z
2365,dsgdb9nsd_000230,0,C,0.087038,1.394377,0.008724
2366,dsgdb9nsd_000230,1,O,-0.030017,-0.008995,0.002400
2367,dsgdb9nsd_000230,2,C,-1.350280,-0.574676,0.011941
2368,dsgdb9nsd_000230,3,C,-2.137122,-0.166239,-1.244013
2369,dsgdb9nsd_000230,4,C,-1.088529,-2.084449,0.001286
2370,dsgdb9nsd_000230,5,C,-2.112947,-0.179915,1.287035
2371,dsgdb9nsd_000230,6,H,1.158248,1.613943,-0.000143
2372,dsgdb9nsd_000230,7,H,-0.365542,1.865800,-0.875376
2373,dsgdb9nsd_000230,8,H,-0.348864,1.856390,0.906072
2374,dsgdb9nsd_000230,9,H,-1.558715,-0.392910,-2.144716


### Molecules as SMILES


In [5]:
import pybel

def xyz_to_smiles(fname):    
    mol = next(pybel.readfile("xyz", fname))
    smi = mol.write(format="smi")
    return smi.split()[0].strip()

In [45]:
smi = xyz_to_smiles("./input/structures/dsgdb9nsd_000011.xyz")
print(smi)

CC=O


In [42]:
mol = next(pybel.readfile("xyz", "./input/structures/dsgdb9nsd_000007.xyz"))
for atom in mol:
   # do something with atom
    print(str(atom.idx) + " : " + str(atom.type))
    print(atom.coords)

1 : C3
(-0.0187040036, 1.5255820146, 0.0104328082)
2 : C3
(0.0021037439, -0.0038819102, 0.0019988214)
3 : H
(0.9948727479, 1.9397432357, 0.0029412016)
4 : H
(-0.5420761069, 1.9236106317, -0.8651173523)
5 : H
(-0.5252411248, 1.9141730785, 0.9000239916)
6 : H
(0.5254865372, -0.4019078419, 0.8775439536)
7 : H
(-1.0114765071, -0.4180337979, 0.0095084879)
8 : H
(0.5086261934, -0.3924704005, -0.8876011721)


In [77]:
import pybel

mol = next(pybel.readfile("xyz", "./input/structures/dsgdb9nsd_000011.xyz"))

smarts = pybel.Smarts("[CH3]") 
print(smarts.findall(mol))

[(1,)]


In [73]:
import openbabel

name = 'dsgdb9nsd_000010'

mol = next(pybel.readfile("xyz", "./input/structures/" + name + ".xyz"))

# Define containers
mol_name = [] # This will be our dataframe index
atom_0 = []
atom_1 = []
bond_order = []
bond_length = []
is_in_ring = []
has_aromatic_bond = []
is_primary_amide = []
is_secondary_amide = []
is_tertiary_amide = []
is_ester = []
is_carbonyl = []


bonds_considered = {}
for atom in mol.atoms:
    
    if atom.atomicnum == 1: # ignore H atoms
        continue

    atom_id = atom.idx - 1
    print("Atom index {}".format(atom_id))
    for neighbour_atom in openbabel.OBAtomAtomIter(atom.OBAtom):
        
        neighbour_atom_id = neighbour_atom.GetIdx()-1
        
        key = str(min(atom_id, neighbour_atom_id)) + '_' + str(max(atom_id, neighbour_atom_id))
        if key in bonds_considered:
            continue
        else:
            bonds_considered[key] = 'counted'

        print("  Neighbor index {}".format(neighbour_atom_id))
        bond = mol.OBMol.GetBond(atom.idx, neighbour_atom.GetIdx())
        
        mol_name.append(name)
        atom_0.append(neighbour_atom_id)
        atom_1.append(atom_id)
        bond_order.append(bond.GetBO())
        bond_length.append(bond.GetLength())
        
        is_in_ring.append(bond.IsInRing())
        has_aromatic_bond.append(bond.IsAromatic())
        is_primary_amide.append(bond.IsPrimaryAmide())
        is_secondary_amide.append(bond.IsSecondaryAmide())
        is_tertiary_amide.append(bond.IsTertiaryAmide())
        is_ester.append(bond.IsEster())
        is_carbonyl.append(bond.IsCarbonyl())
        print("      Bond {}, length {}, order {}".format(bond.GetIdx(), bond.GetLength(), bond.GetBO()))

Atom index 0
  Neighbor index 4
      Bond 0, length 1.0933192240105922, order 1
  Neighbor index 1
      Bond 2, length 1.4569296189663543, order 1
  Neighbor index 3
      Bond 3, length 1.0933172562791795, order 1
  Neighbor index 5
      Bond 4, length 1.093305764923213, order 1
Atom index 1
  Neighbor index 2
      Bond 1, length 1.1550085723365782, order 3
Atom index 2


In [116]:
mol = next(pybel.readfile("xyz", "./input/structures/dsgdb9nsd_000230.xyz"))

smarts = pybel.Smarts("[OH0]") 
print(smarts.findall(mol))


[(2,)]


In [123]:
# Use SMARTS to fill out these arrays
mol = next(pybel.readfile("xyz", "./input/structures/dsgdb9nsd_000013.xyz"))

atom_0_CH3 = []
atom_0_CH2= []
atom_0_CH1= []
atom_0_CH0= []
atom_1_CH3 = []
atom_1_CH2= []
atom_1_CH1= []
atom_1_CH0= []

atom_0_NH2 = []
atom_0_NH1= []
atom_0_NH0= []
atom_1_NH2 = []
atom_1_NH1= []
atom_1_NH0= []

atom_0_OH1 = []
atom_0_OH0= []
atom_1_OH1 = []
atom_1_OH0= []

smarts_CH3 = pybel.Smarts("[CH3]") 
smarts_CH2 = pybel.Smarts("[CH2]") 
smarts_CH1 = pybel.Smarts("[CH1]") 
smarts_CH0 = pybel.Smarts("[CH0]") 

smarts_NH2 = pybel.Smarts("[NH2]") 
smarts_NH1 = pybel.Smarts("[NH1]") 
smarts_NH0 = pybel.Smarts("[NH0]") 

smarts_OH1 = pybel.Smarts("[OH1]") 
smarts_OH0 = pybel.Smarts("[OH0]") 

mol_indexes_of_CH3 = smarts_CH3.findall(mol)
mol_indexes_of_CH2 = smarts_CH2.findall(mol)
mol_indexes_of_CH1 = smarts_CH1.findall(mol)
mol_indexes_of_CH0 = smarts_CH0.findall(mol)

mol_indexes_of_NH2 = smarts_NH2.findall(mol)
mol_indexes_of_NH1 = smarts_NH1.findall(mol)
mol_indexes_of_NH0 = smarts_NH0.findall(mol)

mol_indexes_of_OH1 = smarts_OH1.findall(mol)
mol_indexes_of_OH0 = smarts_OH0.findall(mol)

print(f'mol_indexes_of_CH3: {mol_indexes_of_CH3}')
print(f'mol_indexes_of_CH2: {mol_indexes_of_CH2}')
print(f'mol_indexes_of_CH1: {mol_indexes_of_CH1}')
print(f'mol_indexes_of_CH0: {mol_indexes_of_CH0}')

print(f'mol_indexes_of_NH2: {mol_indexes_of_NH2}')
print(f'mol_indexes_of_NH1: {mol_indexes_of_NH1}')
print(f'mol_indexes_of_NH0: {mol_indexes_of_NH0}')

print(f'mol_indexes_of_OH1: {mol_indexes_of_OH1}')
print(f'mol_indexes_of_OH0: {mol_indexes_of_OH0}')




mol_indexes_of_CH3: [(1,), (3,)]
mol_indexes_of_CH2: [(2,)]
mol_indexes_of_CH1: []
mol_indexes_of_CH0: []
mol_indexes_of_NH2: []
mol_indexes_of_NH1: []
mol_indexes_of_NH0: []
mol_indexes_of_OH1: []
mol_indexes_of_OH0: []


In [74]:
print(len(mol_name))
print(len(atom_0))
print(len(atom_1))
print(len(bond_order))
print(len(bond_length))


df = pd.DataFrame({'molecule_name': mol_name,
                    'atom_0': atom_0,
                    'atom_1': atom_1,
                    'bond_order': bond_order,
                    'bond_length': bond_length,
                    'is_in_ring': is_in_ring,
                    'has_aromatic_bond': has_aromatic_bond,                  
                    'is_primary_amide': is_primary_amide,
                    'is_secondary_amide': is_secondary_amide,
                    'is_tertiary_amide': is_tertiary_amide,
                    'is_ester': is_ester,
                    'is_carbonyl': is_carbonyl,
                  })
    
df = df.sort_values(['molecule_name', 'atom_0', 'atom_1']).reset_index(drop=True)    

df.head(20)

5
5
5
5
5


,molecule_name,atom_0,atom_1,bond_order,bond_length,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl
0,dsgdb9nsd_000010,1,0,1,1.456930,False,False,False,False,False,False,False
1,dsgdb9nsd_000010,2,1,3,1.155009,False,False,False,False,False,False,False
2,dsgdb9nsd_000010,3,0,1,1.093317,False,False,False,False,False,False,False
3,dsgdb9nsd_000010,4,0,1,1.093319,False,False,False,False,False,False,False
4,dsgdb9nsd_000010,5,0,1,1.093306,False,False,False,False,False,False,False


### Functional Groups and other features
#### List of Functional Groups we will classify by

In [73]:
import openbabel

mol = next(pybel.readfile("xyz", "./input/structures/dsgdb9nsd_000230.xyz"))
obmol = mol.OBMol


In [74]:
# Aromatic
print(obmol.GetAtom(1).IsAromatic())

False


In [75]:
# In ring
print(obmol.GetAtom(1).IsInRing())

False


In [77]:
print(obmol.GetAtom(1).HasSingleBond())


True


In [78]:
print(obmol.GetAtom(1).HasDoubleBond())

False


In [79]:
print(obmol.GetAtom(1).HasAromaticBond())

False


In [ ]:
print(obmol.GetAtom(1).HasAromaticBond())